In [1]:
!pip install gradio

In [2]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!pip install gensim
!pip install optuna
!pip install sklearn
!pip install scikit-learn

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vumin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vumin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vumin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [3]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from string import digits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import gensim
import gensim.corpora as corpora
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from gensim.utils import simple_preprocess
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
import optuna
from keras.initializers import Constant
from keras.models import Model
from keras.layers import *

In [4]:
df = pd.read_csv("data.csv")
df.drop_duplicates(inplace=True)
# Remove "Enlarge this image" from the content column
df['content'] = df['content'].str.replace('Enlarge this image', '')
df['content'] = df['content'].str.replace('hide caption', '')
df['content'] = df['content'].str.replace('toggle caption', '')
df['content'] = df['content'].str.replace('caption toggle', '')
# Display the DataFrame with the modified content column

In [5]:
df.category.value_counts()

sports      349
science     349
business    349
politics    349
health      349
Name: category, dtype: int64

In [6]:
df = df.dropna()
df = pd.DataFrame(df)
def remove_url(data):
    url_removed = []
    for line in data:
        url_removed.append(re.sub('http[s]?://\S+', '', line))
    return url_removed

df = df.apply(remove_url)
def remove_hashtag(data):
    hashtag_removed = []
    translator = str.maketrans('#', ' '*len('#'), '')
    for line in data:
        hashtag_removed.append(line.translate(translator))
    return hashtag_removed

df = df.apply(remove_hashtag)
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["title"]= df["title"].apply(lambda text: remove_punctuation(text))
df["content"]= df["content"].apply(lambda text: remove_punctuation(text))
df["title"] = df["title"].apply(lambda x: re.sub(r"[–”“—’‘]", "", x))
df["content"] = df["content"].apply(lambda x: re.sub(r"[–”“—’‘]", "", x))
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
concec = '“'
remove_punctuation(concec)
concec
contractions_dict = {"ain't": "are not","'s":" is","aren't": "are not", "n't" : "not"}
# Regular expression for finding contractions
# "'d" :"had or would"
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions in the reviews
df['content']=df['content'].apply(lambda x:expand_contractions(x))
df['title']=df['title'].apply(lambda x:expand_contractions(x))
df['title'] = df['title'].str.lower()
df['content'] = df['content'].str.lower()
def tokenize_sentence(data):
  tokenized_docs = []

  for line in data:
    tokenized_docs.append(word_tokenize(line))

  return tokenized_docs

df = df.apply(tokenize_sentence)

In [7]:
from nltk.stem import WordNetLemmatizer
def remove_digits(data):
    digit_removed = []

    for doc in data:
        temp = []
        for word in doc:
            # Check if the word contains only digits
            if not word.isdigit():
                temp.append(word)
        digit_removed.append(temp)

    return digit_removed

df = df.apply(remove_digits)
def remove_stopwords(data):

  stopword_removed = []

  stop_words = set(stopwords.words('english'))
  more_stop = ["say", "said", "says", "u"]
  for word in more_stop:
    stop_words.add(word)

  for doc in data:
    temp = []
    for word in doc:
      if word not in stop_words:
        temp.append(word)

    stopword_removed.append(temp)

  return stopword_removed

df = df.apply(remove_stopwords)
def apply_stemmer(data):
  stemmed_docs = []

  stemmer = PorterStemmer()

  for doc in data:
    stemmed_docs.append([stemmer.stem(plural) for plural in doc])

  return stemmed_docs

df = df.apply(apply_stemmer)
def lemmatize_words(data):
    lemmatized_docs = []
    lemmatizer = WordNetLemmatizer()
    for doc in data:
        lemmatized_docs.append([lemmatizer.lemmatize(word) for word in doc])
    return lemmatized_docs
df = df.apply(lemmatize_words)


In [8]:
df = df.drop("link", axis=1)

In [9]:
df['combined_text'] = df.apply(lambda row: row['title'] + row['content'], axis=1)
data_words = df['combined_text']
data_words[:1]
df['category'] = df['category'].apply(lambda x: x[0])
label_encoder = LabelEncoder()

df['label'] = label_encoder.fit_transform(df['category'])

df.label.value_counts()

0    344
1    344
3    343
4    340
2    320
Name: label, dtype: int64

In [10]:
combined_paragraphs = []
# create a full paragraph of new
for index, row in df.iterrows():
    title_text = ' '.join(row['title'])
    content_text = ' '.join(row['content'])
    combined_text = title_text + ' ' + content_text
    combined_paragraphs.append(combined_text)

df['paragraph'] = combined_paragraphs

Word2Vec Embedding

In [11]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Define data
paragraphs = df['paragraph']
tokenized_paragraphs = [simple_preprocess(paragraph) for paragraph in paragraphs]

# Define parameters for Word2Vec model
vector_size = 300
window = 5
min_count = 5


# Train Word2Vec model with specified parameters
word2vec_model = Word2Vec(sentences=tokenized_paragraphs, vector_size=vector_size, window=window, min_count=min_count, sg=0)

# Function to obtain paragraph embeddings using the trained Word2Vec model
def get_paragraph_embedding(paragraph, model):
    # Tokenize and preprocess the paragraph
    tokens = simple_preprocess(paragraph)

    # Filter out tokens that are not in the vocabulary of the Word2Vec model
    tokens_in_vocab = [token for token in tokens if token in model.wv]

    # If no tokens are in the vocabulary, return None
    if not tokens_in_vocab:
        return None

    # Calculate the average word embedding for the tokens in the paragraph
    paragraph_embedding = np.mean([model.wv[token] for token in tokens_in_vocab], axis=0)
    return paragraph_embedding

# Apply the function to each paragraph in the DataFrame
df['skip_gram'] = df['paragraph'].apply(lambda x: get_paragraph_embedding(x, word2vec_model))
print(df['skip_gram'][0])  # Print embedding for the first paragraph

[ 4.20306586e-02  1.63722426e-01 -1.92282144e-02  1.61619410e-02
  1.00160919e-01 -2.10407540e-01  8.89470577e-02  4.05006170e-01
  8.62693638e-02 -5.65641560e-03 -2.54986975e-02 -1.46414369e-01
 -1.34012178e-01 -4.57860567e-02 -2.25958109e-01 -1.16441205e-01
  1.96219176e-01  6.19822592e-02 -1.32391965e-02 -1.04351662e-01
 -1.22760244e-01  7.77166486e-02  5.90593889e-02  4.85615470e-02
  2.68439740e-01  3.93780097e-02 -1.74148008e-01  2.85828300e-02
  5.09759877e-03 -1.40807867e-01 -1.17613576e-01 -6.71308153e-05
  2.53947303e-02 -1.46879122e-01  3.46234255e-02 -1.52096450e-02
  1.71534106e-01 -3.01567703e-01  4.97936606e-02 -3.03444322e-02
 -3.63656543e-02  9.03819576e-02 -3.39686945e-02 -3.63485254e-02
  1.21414363e-01  1.29253268e-01 -1.30038019e-02  1.74925566e-01
 -3.36922407e-02  2.57914096e-01  1.51424751e-01 -3.04229558e-02
 -2.87515014e-01  1.02221355e-01 -7.17159435e-02  1.53629512e-01
  1.55438170e-01  3.41635710e-03  6.85694888e-02 -3.94478440e-03
 -4.92684096e-02 -1.09788

In [12]:
x_train, x_test, y_train, y_test = train_test_split(df['skip_gram'], df['label'], test_size=500, shuffle=False)
x_train = np.array([np.array(x) for x in x_train])
x_test = np.array([np.array(x) for x in x_test])
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1191, 300) (500, 300) (1191,) (500,)


In [13]:

# Instantiate SVM model with specified hyperparameters
svm_model = SVC(C=0.4, kernel='linear', gamma='scale')

# Fit the model to the training data
svm_model.fit(x_train, y_train)

# Make predictions on test data
y_pred = svm_model.predict(x_test)

# Calculate accuracy on test data
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on Test Data:", accuracy)

Accuracy on Test Data: 0.832


In [14]:
import gradio as gr
def make_prediction(text):
    tokens = simple_preprocess(text)
    tokens_in_vocab = [token for token in tokens if token in word2vec_model.wv]
    embedding = get_paragraph_embedding(text, word2vec_model)
    new_text_embedding = np.mean([word2vec_model.wv[token] for token in tokens_in_vocab], axis=0)
    new_text_embedding = [new_text_embedding]
    predictions = svm_model.predict(new_text_embedding)  # Make predictions
    if predictions == 4:
        return "Sport"
    elif predictions == 3:
        return "Science"
    elif predictions == 2:
        return "Politics"
    elif predictions == 1:
        return "Health"
    else:
        return "Business"
    return predictions

news = gr.Textbox()
output = gr.Textbox()
app = gr.Interface(
    fn=make_prediction,
    inputs="text",
    outputs="text",
)
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


c:\Users\vumin\miniconda3\envs\tonlien\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\vumin\miniconda3\envs\tonlien\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Traceback (most recent call last):
  File "c:\Users\vumin\miniconda3\envs\tonlien\lib\site-packages\gradio\queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
  File "c:\Users\vumin\miniconda3\envs\tonlien\lib\site-packages\gradio\route_utils.py", line 258, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\vumin\miniconda3\envs\tonlien\lib\site-packages\gradio\blocks.py", line 1710, in process_api
    result = await self.call_function(
  File "c:\Users\vumin\miniconda3\envs\tonlien\lib\site-packages\gradio\blocks.py", line 1250, in call_function
    predictio